# Dirty Dataset
This example illustrate how to perform meta-blocking on a dirty dataset (data deduplication), so when we have one dataset that contains duplicates.

In [1]:
import sparker
import random

In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

2022-04-03 00:09:59,265 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-04-03 00:09:59,265 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-03 00:09:59,526 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-03 00:09:59,942 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Load the data
sparkER provides wrappers to load CSV and JSON files.

*real_id_field* is the field that contains the identifier of the record.

In [6]:
# Profiles contained in the first dataset
profiles = sparker.JSONWrapper.load_profiles('file:///home/aditi/BigData/AAI_Project/dataset/4_CSV/csv_json/myjson.json', real_id_field = "Id")

Let's visualize a profile to check if they are correctly loaded

In [7]:
print(profiles.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'emp_name', 'value': 'jethalal'}, {'key': 'email', 'value': 'jethalal@yahoo.com'}, {'key': 'job_profile', 'value': 'GET'}, {'key': 'age', 'value': '50'}, {'key': 'gender', 'value': 'm'}], 'original_id': '1', 'source_id': 0}


In [8]:
profiles.take(5)
profiles.toDF().show(1364)

+----------------------+-----------+----------+---------+
|            attributes|original_id|profile_id|source_id|
+----------------------+-----------+----------+---------+
|  [{emp_name, jetha...|          1|         0|        0|
|  [{emp_name, dayab...|          2|         1|        0|
|  [{emp_name, sodhi...|          3|         2|        0|
|  [{emp_name, madha...|          4|         3|        0|
|  [{emp_name, aatma...|          5|         4|        0|
|  [{emp_name, bhide...|          6|         5|        0|
|  [{emp_name, tarak...|          7|         6|        0|
|  [{emp_name, mehta...|          8|         7|        0|
|  [{emp_name, tappu...|          9|         8|        0|
|  [{emp_name, hathi...|         10|         9|        0|
|  [{emp_name, golya...|         11|        10|        0|
|  [{emp_name, sonu}...|         12|        11|        0|
|  [{emp_name, jaysh...|         13|        12|        0|
|  [{emp_name, kiran...|         14|        13|        0|
|  [{emp_name,

Extract the max id (it will be used later)

In [9]:
# Max profile id
max_profile_id = profiles.map(lambda profile: profile.profile_id).max()

In [10]:
max_profile_id

30

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [11]:
# # Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# # respectively the id of profiles of the first dataset and the id of profiles of the second dataset
# gt = sparker.CSVWrapper.load_groundtruth('../datasets/dirty/cora/groundtruth.csv', 'id1', 'id2')

In [12]:
# # Converts the groundtruth by replacing original IDs with those given by Spark
# new_gt = sparker.Converters.convert_groundtruth(gt, profiles)

In [13]:
# # We can explore some pairs
# random.sample(new_gt, 2)

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [14]:
blocks = sparker.Blocking.create_blocks(profiles, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 118


Let's continue by using the standard token blocking

In [15]:
blocks = sparker.Blocking.create_blocks(profiles)
print("Number of blocks",blocks.count())

Number of blocks 16


## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [16]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [17]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [18]:
# recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt)

# print("Recall", recall)
# print("Precision", precision)
# print("Number of comparisons", cmp_n)

## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [19]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [20]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
# gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [30]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
#                           groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 7


### Reciprocal Weighted Node Pruning

In [22]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 7


### Weighted Edge Pruning

In [23]:
results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 1


### Cardinality Node Pruning

In [27]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 7


### Reciprocal Cardinality Node Pruning

In [28]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 1


### Cardinality Edge Pruning

In [29]:
results = sparker.CEP.cep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 6


## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [31]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(10, 19, 1),
 (6, 17, 1),
 (21, 25, 1),
 (5, 25, 1),
 (5, 21, 1),
 (29, 30, 2),
 (3, 13, 1)]

In [32]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          1|            0|       [{10, 19, 1}]|
|          0|            0|                  []|
|          1|            0|        [{6, 17, 1}]|
|          0|            0|                  []|
|          1|            0|       [{21, 25, 1}]|
|          2|            0|[{5, 25, 1}, {5, ...|
|          0|            0|                  []|
|          1|            0|       [{29, 30, 2}]|
|          1|            0|        [{3, 13, 1}]|
|          0|            0|                  []|
|          0|            0|                  []|
+-----------+-------------+--------------------+



In [33]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [34]:
max_by_group.take(10)

[(10, 19, 1), (6, 17, 1), (21, 25, 1), (5, 25, 1), (29, 30, 2), (3, 13, 1)]

In [35]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0   1  2
0  10  19  1
1   6  17  1
2  21  25  1
3   5  25  1
4  29  30  2
5   3  13  1

In [36]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0    10    19       1
1     6    17       1
2    21    25       1
3     5    25       1
4    29    30       2

In [37]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id  N_id  weight
0    10    19       1
1     6    17       1
2    21    25       1
3     5    25       1
4    29    30       2
5     3    13       1

In [36]:

new_df = n[(n['weight']>30)]
new_df

P_id   N_id  weight
0          0  11231      50
1      20512  20516      47
3      16384  16979      49
6      16480  16483      50
7       8304  16439      47
...      ...    ...     ...
38502  29935  29949      76
38526  27599  27651      57
38527  25775  27651      57
38566   7103   8313      31
38568   8607  16526      31

[28420 rows x 3 columns]

In [38]:
new_df = n.sort_values(by=['P_id'])

In [39]:
new_df.head(20)

P_id  N_id  weight
5     3    13       1
3     5    25       1
1     6    17       1
0    10    19       1
2    21    25       1
4    29    30       2

In [39]:
col = ["P_id","N_id","weight"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+-----+------+
|P_id| N_id|weight|
+----+-----+------+
|   0|11231|    50|
|   4|   10|    68|
|  10| 7953|    68|
|  11|   12|    45|
|  12|   14|    45|
|  14|21816|    45|
|  15|13216|    47|
|  17|   18|    50|
|  18|12455|    50|
|  21|   22|   123|
|  22|   24|   123|
|  23|   24|   123|
|  25|   31|    58|
|  26|   27|    44|
|  27|   28|    44|
|  28|18850|    44|
|  29|15104|    86|
|  31|   34|    58|
|  32|18840|    86|
|  34|   36|    58|
+----+-----+------+
only showing top 20 rows



In [40]:
nid_df = pd.DataFrame(new_df.iloc[:,1])
nid_df

N_id
0      11231
9839      10
24320   7953
26735     12
29135     14
...      ...
33382  38592
6914   38595
13543  38598
15972  38599
18396  38600

[28420 rows x 1 columns]

In [41]:
ss = SparkSession(sc)


In [42]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .json('hdfs://localhost:9000/Final_Project/sample/Df2.json')
df1.show(5)

+-----------+--------------------+-----+--------------------+-------------------+-----------+----------------------+--------------------+
|brand_right|   description_right|   id|   identifiers_right|keyValuePairs_right|price_right|specTableContent_right|         title_right|
+-----------+--------------------+-----+--------------------+-------------------+-----------+----------------------+--------------------+
|       null|vigile su hogar d...|61622|[{null, null, nul...|               null|       null|  versi n descripci...|outdoor hd wirele...|
|       null|the dcs 6010l 2 m...|61623|[{null, null, nul...|               null|       null|  version descripti...|2 megapixel panor...|
|       null|the dcs 6010l 2 m...|61626|[{null, null, nul...|               null|       null|  version descripti...|2 megapixel panor...|
|       null|vigile su hogar d...|61628|[{null, null, nul...|               null|       null|  versi n descripci...|outdoor hd wirele...|
|       null|the dcs 2330l hd ...|

In [127]:
#rdd_df = df1.rdd.zipWithIndex()

In [128]:
# rdd_df.take(1)

NameError: name 'rdd_df' is not defined

In [ ]:
#df1 = rdd_df.toDF(sampleRatio=0.01)

In [43]:
# df1.show()

In [44]:
# df1 = df1.withColumn('brand_left', df1['_1'].getItem("brand_left"))
# df1 = df1.withColumn('description_left', df1['_1'].getItem("description_left"))
# df1 = df1.withColumn('id_left', df1['_1'].getItem("id_left"))
# df1 = df1.withColumn('identifiers_left', df1['_1'].getItem("identifiers_left"))
# df1 = df1.withColumn('keyValuePairs_left', df1['_1'].getItem("keyValuePairs_left"))
# df1 = df1.withColumn('specTableContent_left', df1['_1'].getItem("specTableContent_left"))
# df1 = df1.withColumn('title_left', df1['_1'].getItem("title_left"))
# df1 = df1.withColumn('id', df1['_1'].getItem("id"))


# df1 = df1.withColumnRenamed("_2" , "new_id_1")

In [45]:
# df1 = df1.drop("_1")
# df1.show()

In [46]:
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [47]:
pd_common_df = common_df.toPandas()
pd_common_df

brand_right                                  description_right  \
0             sandisk  description sandisk extreme microsdhc 64gb typ...   
1               canon  this is considered the standard lens for use w...   
2      action outdoor  description action outdoor floating hand grip ...   
3                ksix  product description ksix selfie extension for ...   
4                None  for purchase inquires please call 416 977 9711...   
...               ...                                                ...   
13541       "SanDisk"                                               None   
13542   hp enterprise  description hp ethernet 1gb 4 port 331t adapte...   
13543            None                                               None   
13544   hp enterprise  description hp intel xeon e5506 dl380 g6 2 13g...   
13545            None                                               None   

          id                                  identifiers_right  \
0       8440  [(None, None, None, None, None, None, None, [1...   
1       8484  [(None, None, None, None, None, [2514a002], No...   
2       9458  [(None, None, None, None, None, None, None, [1...   
3       9715  [(None, None, None, None, None, None, None, [1...   
4       9968  [(None, None, None, None, None, None, None, [8...   
...      ...                                                ...   
13541  33096  [(None, None, None, None, None, None, None, [s...   
13542  33951  [(None, None, None, None, None, [647594b21], N...   
13543  34254  [(None, None, None, None, None, [sda1064gb], N...   
13544  34584  [(None, None, None, None, None, None, None, [4...   
13545  35266  [(None, None, None, None, None, [bx80671i76800...   

                                     keyValuePairs_right price_right  \
0                                                   None        None   
1                                                   None        None   
2                                                   None        None   
3                                                   None        None   
4      (None, None, None, None, None, None, None, Non...        None   
...                                                  ...         ...   
13541                                               None   usd 56 43   
13542  (None, None, None, None, None, None, None, Non...  usd 229 95   
13543  (None, None, None, None, None, None, None, Non...        None   
13544  (None, None, None, None, None, None, None, Non...        None   
13545  (None, None, None, None, None, None, None, Non...        None   

                                  specTableContent_right  \
0                                                   None   
1                                                   None   
2                                                   None   
3                                                   None   
4      manufacturer canon sku 8806a002 our price 849 ...   
...                                                  ...   
13541                                               None   
13542  specifications category server adapter sub cat...   
13543  categorie geheugenkaarten merk kingston produc...   
13544  specifications category proliant processor sub...   
13545  quick specs compatibility desktop process tech...   

                                             title_right   P_id   N_id  weight  
0      sandisk extreme microsdhc 64gb type 10 comprar...   8440   9084      53  
1      canon ef 50mm f 1 8 ii camera lens fixed disco...   8484  10148     150  
2      action outdoor floating hand grip bobber compr...   9458  17198      34  
3      ksix selfie extension for gopro and sport came...   9715  22400      62  
4      canon ef 17 40mm f4l usm digital slr camera ca...   9968  16025      70  
...                                                  ...    ...    ...     ...  
13541  "200GB Ultra UHS-I microSDXC Memory Card (Clas...  33096  34984     207  
13542  647594 b21 hp ethernet 1gb 4 port 331t adapter...  

In [48]:
pd_common_df = pd_common_df.sort_values(by=['P_id'])
pd_common_df

brand_right                                  description_right  \
6149    hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...   
7168             None                                               None   
6711             asus  support for x series intel core processors sli...   
7026             None                                               None   
7273             None                                               None   
...               ...                                                ...   
4948             None                                               None   
5356             None                                               None   
4591  western digital                                               None   
4740               wd  2tb capacity 3 5 inch sata 6gb s interface 540...   
4685               wd  2tb capacity 3 5 inch sata 6gb s interface 540...   

         id                                  identifiers_right  \
6149      0  [(None, None, None, None, None, [495906b21], N...   
7168      4  [(None, None, None, None, None, [primex299delu...   
6711     10  [(None, None, None, None, None, b0ty0m0eay...   
7026     11  [(None, None, None, None, None, [primex299delu...   
7273     14  [(None, None, None, None, None, [h81mdgsr20,  ...   
...     ...                                                ...   
4948  38578  [(None, None, None, None, None, [wd20purz], No...   
5356  38588  [(None, None, None, None, None, [wd20purz], No...   
4591  38589  [(None, None, None, None, None, None, None, [w...   
4740  38597  [(None, None, None, None, None, [wd20purz], No...   
4685  38598  [(None, None, None, None, None, [wd20purz], No...   

                                    keyValuePairs_right price_right  \
6149  (None, None, None, None, None, None, None, Non...  usd 213 85   
7168  (None, None, None, None, None, None, None, Non...        None   
6711                                               None        None   
7026  (None, None, None, None, None, None, None, Non...        None   
7273  (None, None, None, None, None, None, None, Non...        None   
...                                                 ...         ...   
4948  (None, None, None, None, None, None, None, Non...        None   
5356  (None, None, None, None, None, None, None, Non...        None   
4591                                               None        None   
4740                                               None        None   
4685                                               None        None   

                                 specTableContent_right  \
6149  specifications category proliant processor sub...   
7168  categorie moederborden merk asus product asus ...   
6711                                               None   
7026  categorie moederborden merk asus product asus ...   
7273  categorie moederborden merk asrock product asr...   
...                                                 ...   
4948  categorie interne harde schijven merk wd produ...   
5356  categorie interne harde schijven merk wd produ...   
4591  r n kodu wd40ezrz ba lant aray z sata iii kapa...   
4740                                               None   
4685                                               None   

                                            title_right   P_id   N_id  weight  
6149        null , 495906 b21 hp x5560 2 80ghz ml350 g6      0  11231      50  
7168            asus prime x299 deluxe prijzen tweakers      4     10      68  
6711  asus prime x299 deluxe socket 2066 intel atx m...     10   7953      68  
7026            asus prime x299 deluxe prijzen tweakers     11     12      45  
7273              asrock h81m dgs r2 0 prijzen tweakers     14  21816      45  
...                                                 ...    ...    ...     ...  
4948            wd purple wd20purz 2tb prijzen tweakers  38578  38579      33  
5356            wd purple wd20purz 2tb prijzen tweakers  38588  38590      33  
4591  wd 4tb 3 5 blue 64mb

In [49]:
pd_common_df = pd_common_df[~pd_common_df.id.isin(nid_df.N_id)]
pd_common_df

brand_right                                  description_right     id  \
6149  hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...      0   
7168           None                                               None      4   
7026           None                                               None     11   
7352           None                                               None     15   
6553           None                                               None     17   
...             ...                                                ...    ...   
6060  hp enterprise  description hp flexfabric 10gb 2 port 554flr s...  38395   
4832           None                                               None  38397   
4958           None                                               None  38399   
5333        sandisk                                               None  38414   
5941  hp enterprise  description intel xeon e5507 ml350 g6 2 26ghz ...  38481   

                                      identifiers_right  \
6149  [(None, None, None, None, None, [495906b21], N...   
7168  [(None, None, None, None, None, [primex299delu...   
7026  [(None, None, None, None, None, [primex299delu...   
7352  [(None, None, None, None, None, [h81mdgsr20,  ...   
6553  [(None, None, None, None, None, [h81mdgsr20,  ...   
...                                                 ...   
6060  [(None, None, None, None, None, [629142b21], N...   
4832  [(None, None, None, None, None, [603608b21], N...   
4958  [(None, None, None, None, None, [stea1000400],...   
5333  [(None, None, None, None, None, None, None, [s...   
5941  [(None, None, None, None, None, None, None, [6...   

                                    keyValuePairs_right     price_right  \
6149  (None, None, None, None, None, None, None, Non...      usd 213 85   
7168  (None, None, None, None, None, None, None, Non...            None   
7026  (None, None, None, None, None, None, None, Non...            None   
7352  (None, None, None, None, None, None, None, Non...            None   
6553  (None, None, None, None, None, None, None, Non...            None   
...                                                 ...             ...   
6060  (None, None, None, None, None, None, None, Non...  454 94 cad cad   
4832  (None, None, None, None, None, None, None, Non...            None   
4958  (None, None, None, None, None, None, None, Non...            None   
5333                                               None            None   
5941  (None, None, None, None, None, None, None, Non...            None   

                                 specTableContent_right  \
6149  specifications category proliant processor sub...   
7168  categorie moederborden merk asus product asus ...   
7026  categorie moederborden merk asus product asus ...   
7352  categorie moederborden merk asrock product asr...   
6553  categorie moederborden merk asrock product asr...   
...                                                 ...   
6060  specifications category server adapter sub cat...   
4832  categorie processors merk hp product hp intel ...   
4958  categorie externe harde schijven merk seagate ...   
5333  r n kodu sdcz33 016g b35 garanti s resi 2 y l ...   
5941  specifications category proliant processor sub...   

                                            title_right   P_id   N_id  weight  
6149        null , 495906 b21 hp x5560 2 80ghz ml350 g6      0  11231      50  
7168            asus prime x299 deluxe prijzen tweakers      4     10      68  
7026            asus prime x299 deluxe prijzen tweakers     11     12      45  
7352              asrock h81m dgs r2 0 prijzen tweakers     15  13216      47  
6553              asrock h81m dgs r2 0 prijzen tweakers     17     18      50  
...                                                 ...    ...    ...     ...  
6060  629142 b21 hp flexfabric 10gb 2 port 554flr ad...  38395  38569      91  
4832               hp intel xeon x7560 prijzen tweakers  38397  38569      9

In [50]:
pd_common_df = pd_common_df.sort_values(by=['id'])
pd_common_df

brand_right                                  description_right     id  \
6149  hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...      0   
7168           None                                               None      4   
7026           None                                               None     11   
7352           None                                               None     15   
6553           None                                               None     17   
...             ...                                                ...    ...   
6060  hp enterprise  description hp flexfabric 10gb 2 port 554flr s...  38395   
4832           None                                               None  38397   
4958           None                                               None  38399   
5333        sandisk                                               None  38414   
5941  hp enterprise  description intel xeon e5507 ml350 g6 2 26ghz ...  38481   

                                      identifiers_right  \
6149  [(None, None, None, None, None, [495906b21], N...   
7168  [(None, None, None, None, None, [primex299delu...   
7026  [(None, None, None, None, None, [primex299delu...   
7352  [(None, None, None, None, None, [h81mdgsr20,  ...   
6553  [(None, None, None, None, None, [h81mdgsr20,  ...   
...                                                 ...   
6060  [(None, None, None, None, None, [629142b21], N...   
4832  [(None, None, None, None, None, [603608b21], N...   
4958  [(None, None, None, None, None, [stea1000400],...   
5333  [(None, None, None, None, None, None, None, [s...   
5941  [(None, None, None, None, None, None, None, [6...   

                                    keyValuePairs_right     price_right  \
6149  (None, None, None, None, None, None, None, Non...      usd 213 85   
7168  (None, None, None, None, None, None, None, Non...            None   
7026  (None, None, None, None, None, None, None, Non...            None   
7352  (None, None, None, None, None, None, None, Non...            None   
6553  (None, None, None, None, None, None, None, Non...            None   
...                                                 ...             ...   
6060  (None, None, None, None, None, None, None, Non...  454 94 cad cad   
4832  (None, None, None, None, None, None, None, Non...            None   
4958  (None, None, None, None, None, None, None, Non...            None   
5333                                               None            None   
5941  (None, None, None, None, None, None, None, Non...            None   

                                 specTableContent_right  \
6149  specifications category proliant processor sub...   
7168  categorie moederborden merk asus product asus ...   
7026  categorie moederborden merk asus product asus ...   
7352  categorie moederborden merk asrock product asr...   
6553  categorie moederborden merk asrock product asr...   
...                                                 ...   
6060  specifications category server adapter sub cat...   
4832  categorie processors merk hp product hp intel ...   
4958  categorie externe harde schijven merk seagate ...   
5333  r n kodu sdcz33 016g b35 garanti s resi 2 y l ...   
5941  specifications category proliant processor sub...   

                                            title_right   P_id   N_id  weight  
6149        null , 495906 b21 hp x5560 2 80ghz ml350 g6      0  11231      50  
7168            asus prime x299 deluxe prijzen tweakers      4     10      68  
7026            asus prime x299 deluxe prijzen tweakers     11     12      45  
7352              asrock h81m dgs r2 0 prijzen tweakers     15  13216      47  
6553              asrock h81m dgs r2 0 prijzen tweakers     17     18      50  
...                                                 ...    ...    ...     ...  
6060  629142 b21 hp flexfabric 10gb 2 port 554flr ad...  38395  38569      91  
4832               hp intel xeon x7560 prijzen tweakers  38397  38569      9

In [51]:
pd_common_df.reset_index(inplace = True)
pd_common_df = pd_common_df.drop(["index"], axis=1)

In [52]:
pd_common_df = pd_common_df.drop(["identifiers_right","keyValuePairs_right","P_id","N_id","weight"], axis =1)
pd_common_df

brand_right                                  description_right     id  \
0     hp enterprise  description intel xeon x5560 ml350 g6 2 80ghz ...      0   
1              None                                               None      4   
2              None                                               None     11   
3              None                                               None     15   
4              None                                               None     17   
...             ...                                                ...    ...   
7215  hp enterprise  description hp flexfabric 10gb 2 port 554flr s...  38395   
7216           None                                               None  38397   
7217           None                                               None  38399   
7218        sandisk                                               None  38414   
7219  hp enterprise  description intel xeon e5507 ml350 g6 2 26ghz ...  38481   

         price_right                             specTableContent_right  \
0         usd 213 85  specifications category proliant processor sub...   
1               None  categorie moederborden merk asus product asus ...   
2               None  categorie moederborden merk asus product asus ...   
3               None  categorie moederborden merk asrock product asr...   
4               None  categorie moederborden merk asrock product asr...   
...              ...                                                ...   
7215  454 94 cad cad  specifications category server adapter sub cat...   
7216            None  categorie processors merk hp product hp intel ...   
7217            None  categorie externe harde schijven merk seagate ...   
7218            None  r n kodu sdcz33 016g b35 garanti s resi 2 y l ...   
7219            None  specifications category proliant processor sub...   

                                            title_right  
0           null , 495906 b21 hp x5560 2 80ghz ml350 g6  
1               asus prime x299 deluxe prijzen tweakers  
2               asus prime x299 deluxe prijzen tweakers  
3                 asrock h81m dgs r2 0 prijzen tweakers  
4                 asrock h81m dgs r2 0 prijzen tweakers  
...                                                 ...  
7215  629142 b21 hp flexfabric 10gb 2 port 554flr ad...  
7216               hp intel xeon x7560 prijzen tweakers  
7217  seagate expansion portable 1tb zwart prijzen t...  
7218  sandisk 16gb cruzer fit usb bellek sdcz33 016g...  
7219  null , 601248 b21 hp e5507 2 26ghz ml350 g6 wh...  

[7220 rows x 6 columns]

In [53]:
pd_common_df["new_id"] = [x for x in range(0,len(pd_common_df))]
pd_common_df = pd_common_df.drop(["id"], axis=1)

In [54]:
sp_df2 = spark.createDataFrame(pd_common_df)
sp_df2

DataFrame[brand_right: string, description_right: string, price_right: string, specTableContent_right: string, title_right: string, new_id: bigint]

In [55]:
pd_common_df.to_json('/home/aditi/BigData/AAI_Project/dataset/df2_F2.json')

In [56]:
baseDir = 'hdfs://localhost:9000/Final_Project/sample/DF_op/Df2_F2.json'
sp_df2.write.option("header", True).mode('overwrite').json(baseDir)